## Benchmark

In [ ]:
import os

import torch

from llm_benchmarks.generation import generate_samples
from llm_benchmarks.utils import log_metrics_to_csv

os.environ["WANDB_SILENT"] = "true"

config = {
    "load_in_8bit": True,
    "torch_dtype": torch.float16,
    "temperature": 0.1,
    "min_length": 700,
    "max_length": 700,
    "try_different_lengths": True,
}

model_names = ["databricks/dolly-v2-3b", "databricks/dolly-v2-7b", "databricks/dolly-v2-12b"]
for model_name in model_names:
    config["model_name"] = model_name
    metrics = generate_samples(model_name, config, 20)
    log_metrics_to_csv(model_name, config, metrics, "results")

## Plot

In [ ]:
# Load all csv files that contain 'dolly' in the filename and put into a single dataframe
import glob

import pandas as pd
import plotly.express as px

df = pd.concat([pd.read_csv(f) for f in glob.glob("./results/*dolly*.csv")], ignore_index=True)

# Remove outliers
# df = df[df['tokens_per_second'] < 100]
# df = df[df['output_tokens'] > 500]

# Plot the DataFrame
# colors = {True: 'blue', False: 'red'}
fig = px.scatter(df, x="output_tokens", y="tokens_per_second", color="model_name")
fig.update_traces(marker=dict(size=12))
fig.update_layout(xaxis_title="output_tokens", yaxis_title="tokens_per_second", title="Scatter Plot")
fig.update_layout(title="Dolly-v2: Tokens per Second vs. Output Tokens")

fig.show()

fig.write_image("dolly_compare_8bit_inference.png", format="png", width=900, height=500, scale=5)